In [1]:
from power import *
from power.systems import System3Bus
from power.systems import IEEE14
import pulp as pl

In [2]:
b3 = System3Bus()
b14 = IEEE14()

In [ ]:
# Variáveis e formulação do despacho econômico (linear) usando PuLP
# Escolha a rede a ser despachada: trocar para b3 se preferir o sistema 3-barras
net = b3
generators = net.generators

# (Re)cria o problema como minimização do custo linear
problema = pl.LpProblem("Despacho_Economico_Linear", pl.LpMinimize)

# Variáveis de decisão: potência ativa por gerador em pu
p_gen = {g.id: pl.LpVariable(f"p_g_{g.id}", lowBound=g.p_min, upBound=g.p_max) for g in generators}

# Objetivo: soma dos custos lineares (usa g.cost_b já escalado pelo pb nas propriedades do modelo)
problema += pl.lpSum([g.cost_b * p_gen[g.id] for g in generators]), "Custo_Total"

# Restrição de balanço de potência: geração total = carga total (em pu)
total_load = sum([ld.p for ld in net.loads])
problema += pl.lpSum([p_gen[g.id] for g in generators]) == total_load, "Balanceamento_Potencia"

# Resolver o problema e imprimir resultados
resultado = problema.solve()
print("Status:", pl.LpStatus[problema.status])
print("Custo objetivo (PU):", pl.value(problema.objective))
print("Carga total (pu):", total_load)

# Atualizar os objetos gerador com as decisões e mostrar um resumo (primeiros 30 geradores para não poluir)
for g in generators:
    pg = p_gen[g.id].value()
    print(f"Gen {g.id} @ bus {g.bus.id}: p = {pg:.6f} pu -> {pg * g.pb:.2f} MW (limites [{g.p_min:.6f},{g.p_max:.6f}])")
    # escrever de volta no modelo (opcional) em MW: g.p_input = pg * g.pb

# Total geração calculada (sanity-check)
print("Soma da geração (pu):", sum(p_gen[g.id].value() for g in generators))

Status: Optimal
Custo objetivo (PU): 10000.0
Carga total (pu): 0.1
Gen 1 @ bus 1: p = 0.100000 pu -> 10.00 MW (limites [0.000000,0.150000])
Gen 2 @ bus 2: p = 0.000000 pu -> 0.00 MW (limites [0.000000,0.150000])
Gen 3 @ bus 3: p = 0.000000 pu -> 0.00 MW (limites [0.000000,0.150000])
Gen 1001 @ bus 1: p = 0.000000 pu -> 0.00 MW (limites [0.000000,999.990000])
Gen 1002 @ bus 2: p = 0.000000 pu -> 0.00 MW (limites [0.000000,999.990000])
Gen 1003 @ bus 3: p = 0.000000 pu -> 0.00 MW (limites [0.000000,999.990000])
Soma da geração (pu): 0.1


In [4]:
print(b3.generators)

[Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,0.150], q_range=[-999.99,999.99]),, Generator(id=2, bus=2, p=0.000, q=0.000, p_range=[0.000,0.150], q_range=[-999.99,999.99]),, Generator(id=3, bus=3, p=0.000, q=0.000, p_range=[0.000,0.150], q_range=[-999.99,999.99]),, Generator(id=1001, bus=1, p=0.000, q=0.000, p_range=[0.000,999.990], q_range=[-999.99,999.99]),, Generator(id=1002, bus=2, p=0.000, q=0.000, p_range=[0.000,999.990], q_range=[-999.99,999.99]),, Generator(id=1003, bus=3, p=0.000, q=0.000, p_range=[0.000,999.990], q_range=[-999.99,999.99]),]
